In [1]:
import pandas as pd
import flair

/Users/zulikahlatief/Desktop/personal/NLP/nlp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = flair.models.TextClassifier.load('en-sentiment')

In [3]:
model

TextClassifier(
  (embeddings): TransformerDocumentEmbeddings(
    (model): DistilBertModel(
      (embeddings): Embeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (transformer): Transformer(
        (layer): ModuleList(
          (0-5): 6 x TransformerBlock(
            (attention): MultiHeadSelfAttention(
              (dropout): Dropout(p=0.1, inplace=False)
              (q_lin): Linear(in_features=768, out_features=768, bias=True)
              (k_lin): Linear(in_features=768, out_features=768, bias=True)
              (v_lin): Linear(in_features=768, out_features=768, bias=True)
              (out_lin): Linear(in_features=768, out_features=768, bias=True)
            )
            (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (ffn): 

In [4]:
def get_sentiment(text):
    sentence = flair.data.Sentence(text)
    model.predict(sentence)
    sentiment = sentence.labels[0].score
    return sentiment

In [5]:
df = pd.read_csv('reddit_investing_ner.csv', sep='|')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 804 entries, 0 to 803
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   name           804 non-null    object 
 1   created_utc    804 non-null    float64
 2   subreddit      804 non-null    object 
 3   title          804 non-null    object 
 4   selftext       804 non-null    object 
 5   upvote_ratio   804 non-null    float64
 6   ups            804 non-null    int64  
 7   downs          804 non-null    int64  
 8   score          804 non-null    int64  
 9   organizations  804 non-null    object 
dtypes: float64(2), int64(3), object(5)
memory usage: 62.9+ KB


In [6]:
df['sentiment'] = df['selftext'].apply(get_sentiment)
df.head()

,name,created_utc,subreddit,title,selftext,upvote_ratio,ups,downs,score,organizations,sentiment
0,t3_13qkfat,1.684932e+09,investing,Exploration of the Arbitrage Co-movement Effec...,\n\nThere is very little doubt that exchange-...,0.50,0,0,0,[],0.927749
1,t3_13qiqm6,1.684927e+09,investing,What’s the best artificial intelligence ETF to...,"Hi,\n\ni want to add an AI ETF to my portfolio...",0.20,0,0,0,[],0.823243
2,t3_13qh6aq,1.684922e+09,investing,When will the US debt become a real issue?,"Hello all, \n\nI noticed the USD 31 trillion ...",0.50,0,0,0,[],0.509022
3,t3_13qh3p2,1.684922e+09,investing,I just put 50k on a roboadvisor,I am a (27m) very average basic investor with ...,0.60,1,0,1,[],0.998010
4,t3_13qgtpy,1.684921e+09,investing,"The Impact of Financial Crisis, Behavioral Bia...",In a hypothetical scenario where the financial...,0.22,0,0,0,[],0.988111


In [7]:
df['sentiment']

0      0.927749
1      0.823243
2      0.509022
3      0.998010
4      0.988111
         ...   
799    0.999912
800    0.771912
801    0.971103
802    0.990455
803    0.995066
Name: sentiment, Length: 804, dtype: float64

In [8]:
import ast

In [9]:
df['organizations'] = df['organizations'].apply(lambda x: ast.literal_eval(x))

In [13]:
sentiment = {}

for i, row in df.iterrows():
    score = row['sentiment']
    for org in row['organizations']:
        if org not in sentiment.keys():
            sentiment[org] = {'POSITIVE': [], 'NEGATIVE': []}
        sentiment[org].update(score)

TypeError: 'float' object is not iterable